In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
NSE Option Chain Sentiment — Final (Top-5 S/R + Closest Gaps)
-------------------------------------------------------------
• Reads tickers from symbols.txt
• Prints detailed report per symbol to terminal
• Saves one fresh summary.csv each run (no per-symbol files)
"""

from __future__ import annotations
import os, time, math, re, datetime as dt
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import pandas as pd, requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# ---------------- CONFIG ----------------
class CONFIG:
    SYMBOLS_FILE = "symbols.txt"
    OUTPUT_DIR = "option_chain_outputs"
    EXPIRY_PREF = "auto"
    TOTAL_RETRIES = 3
    BACKOFF_FACTOR = 0.35
    REQUEST_SLEEP_SEC = 1.0

NSE_HOST = "https://www.nseindia.com"
OC_ENDPOINTS = {
    "indices": NSE_HOST + "/api/option-chain-indices?symbol={symbol}",
    "equities": NSE_HOST + "/api/option-chain-equities?symbol={symbol}",
}
HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) Chrome/120 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Referer": NSE_HOST + "/option-chain",
}
MONTH_MAP = dict(JAN=1,FEB=2,MAR=3,APR=4,MAY=5,JUN=6,JUL=7,AUG=8,SEP=9,OCT=10,NOV=11,DEC=12)
DATE_PAT = re.compile(r"(\d{1,2})-([A-Za-z]{3})-(\d{4})")

# ---------- HTTP client ----------
def build_session(retries, backoff):
    s = requests.Session()
    s.headers.update(HEADERS)
    retry = Retry(
        total=retries,
        backoff_factor=backoff,
        status_forcelist=(429,500,502,503,504),
        allowed_methods=("GET","HEAD")
    )
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

class NSEClient:
    def __init__(self):
        self.s = build_session(CONFIG.TOTAL_RETRIES, CONFIG.BACKOFF_FACTOR)
    def boot(self):
        self.s.get(NSE_HOST, timeout=10)
    def fetch(self, sym, seg):
        self.boot()
        url = OC_ENDPOINTS[seg].format(symbol=sym)
        r = self.s.get(url, timeout=15)
        if r.status_code != 200:
            raise RuntimeError(f"{sym}: HTTP {r.status_code}")
        return r.json()

# ---------- utilities ----------
def parse_exp(s):
    m = DATE_PAT.fullmatch(s.strip())
    if m:
        return dt.date(int(m.group(3)), MONTH_MAP[m.group(2).upper()[:3]], int(m.group(1)))
    return dt.date.fromisoformat(s)

def classify_exp(d):
    first_next = (d.replace(day=1) + dt.timedelta(days=32)).replace(day=1)
    last = first_next - dt.timedelta(days=1)
    t = last
    while t.weekday() != 3:  # Thursday
        t -= dt.timedelta(days=1)
    return "monthly" if d == t else "weekly"

def choose_exp(exp, pref):
    if pref not in {"auto","weekly","monthly"}:
        wanted = parse_exp(pref)
        for e in exp:
            if parse_exp(e) == wanted:
                return e
        return exp[0]
    dated = sorted((parse_exp(e), e) for e in exp)
    today = dt.date.today()
    for d,e in dated:
        if d >= today:
            if pref == "auto" or classify_exp(d) == pref:
                return e
    return dated[0][1]

def infer_segment(sym):
    return "indices" if sym.upper() in {"NIFTY","BANKNIFTY","FINNIFTY"} else "equities"

# ---------- core transforms ----------
def to_df(raw):
    recs = raw.get("records", {}).get("data", [])
    rows = []
    for n in recs:
        s = n.get("strikePrice")
        exp = n.get("expiryDate")
        ce = n.get("CE", {})
        pe = n.get("PE", {})
        rows.append(dict(
            strike=s, expiry=exp,
            ce_oi=ce.get("openInterest",0), ce_coi=ce.get("changeinOpenInterest",0),
            pe_oi=pe.get("openInterest",0), pe_coi=pe.get("changeinOpenInterest",0)
        ))
    df = pd.DataFrame(rows)
    for c in ["strike","ce_oi","ce_coi","pe_oi","pe_coi"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0)
    return df.dropna(subset=["expiry"])

@dataclass
class Report:
    symbol:str; segment:str; expiry:str; asof:str
    pcr:float; total_call_oi:int; total_put_oi:int
    delta_call_oi:int; delta_put_oi:int
    call_top5:List[Tuple[int,int]]; put_top5:List[Tuple[int,int]]
    ce_hot:Optional[Tuple[int,int]]; pe_hot:Optional[Tuple[int,int]]
    res_gaps:List[int]; sup_gaps:List[int]
    equilibrium:bool; eq_strike:Optional[int]
    sentiment:str; rationale:str

def _topn(s, n):
    s = s[s > 0].sort_values(ascending=False)
    return [(int(k), int(v)) for k, v in s.head(n).items()]

def _gaps(vals, k=3):
    if len(vals) < 2:
        return []
    u = sorted(set(vals))
    g = [u[i+1]-u[i] for i in range(len(u)-1) if u[i+1] > u[i]]
    return sorted(g)[:k]

def derive_sentiment(pcr, d_put, d_call):
    score, reason = 0, []
    if pcr < 0.8:
        score -= 1; reason.append(f"PCR {pcr:.2f} (<0.80) → bearish tilt")
    elif pcr > 1.2:
        score += 1; reason.append(f"PCR {pcr:.2f} (>1.20) → bullish tilt")
    else:
        reason.append(f"PCR {pcr:.2f} (~neutral)")
    if d_put > d_call:
        score += 1; reason.append(f"ΔPutOI {d_put:,} > {d_call:,} → bullish tilt")
    elif d_call > d_put:
        score -= 1; reason.append(f"ΔCallOI {d_call:,} > {d_put:,} → bearish tilt")
    else:
        reason.append("ΔOI neutral")
    return ("Bullish" if score > 0 else "Bearish" if score < 0 else "Neutral",
            "; ".join(reason))

def analyze(raw, sym, seg, pref):
    recs = raw.get("records", {})
    exp = choose_exp(recs.get("expiryDates", []), pref)
    df = to_df(raw)
    dfe = df[df["expiry"] == exp]
    tce = int(dfe["ce_oi"].sum())
    tpe = int(dfe["pe_oi"].sum())
    pcr = (tpe / tce) if tce else math.inf
    dce = int(dfe["ce_coi"].sum())
    dpe = int(dfe["pe_coi"].sum())
    ce_agg = _topn(dfe.groupby("strike")["ce_oi"].sum(), 5)
    pe_agg = _topn(dfe.groupby("strike")["pe_oi"].sum(), 5)
    ce_flow = dfe.groupby("strike")["ce_coi"].sum().sort_values(ascending=False)
    pe_flow = dfe.groupby("strike")["pe_coi"].sum().sort_values(ascending=False)
    ce_hot = (int(ce_flow.index[0]), int(ce_flow.iloc[0])) if not ce_flow.empty else None
    pe_hot = (int(pe_flow.index[0]), int(pe_flow.iloc[0])) if not pe_flow.empty else None
    sentiment, rationale = derive_sentiment(pcr, dpe, dce)
    eq = False; eqs = None
    if ce_agg and pe_agg and ce_agg[0][0] == pe_agg[0][0]:
        eq = True; eqs = ce_agg[0][0]
    return Report(
        sym, seg, exp, recs.get("timestamp",""), pcr,
        tce, tpe, dce, dpe, ce_agg, pe_agg,
        ce_hot, pe_hot, _gaps([s for s,_ in ce_agg]),
        _gaps([s for s,_ in pe_agg]), eq, eqs, sentiment, rationale
    )

# ---------- formatting ----------
def fmt_pairs(p): return "; ".join(f"{s}:{oi}" for s,oi in p)
def fmt_list(l): return "; ".join(map(str,l))
def build_report(r:Report)->str:
    L=[]
    L.append("="*72)
    L.append(f"Symbol: {r.symbol} | Segment: {r.segment} | Expiry: {r.expiry}")
    L.append(f"As Of : {r.asof}")
    L.append("-"*72)
    L.append(f"Total Call OI: {r.total_call_oi:,}")
    L.append(f"Total Put  OI: {r.total_put_oi:,}")
    L.append(f"PCR        : {r.pcr:.2f}")
    L.append(f"ΔCall OI   : {r.delta_call_oi:,}")
    L.append(f"ΔPut  OI   : {r.delta_put_oi:,}")
    L.append("-"*72)
    L.append("Top-5 Resistances (CE OI): "+fmt_pairs(r.call_top5))
    L.append("Top-5 Supports    (PE OI): "+fmt_pairs(r.put_top5))
    if r.ce_hot:L.append(f"Flow (ΔOI) CE hotspot: {r.ce_hot[0]} | ΔOI: {r.ce_hot[1]:,}")
    if r.pe_hot:L.append(f"Flow (ΔOI) PE hotspot: {r.pe_hot[0]} | ΔOI: {r.pe_hot[1]:,}")
    if r.res_gaps:L.append(f"Closest resistance gaps: {fmt_list(r.res_gaps)}")
    if r.sup_gaps:L.append(f"Closest support gaps   : {fmt_list(r.sup_gaps)}")
    L.append("-"*72)
    if r.equilibrium:L.append(f"⚪ Equilibrium Zone: {r.eq_strike}")
    L.append(f"Sentiment: {r.sentiment}")
    L.append(f"Why      : {r.rationale}")
    L.append("="*72)
    return "\n".join(L)

# ---------- main ----------
if __name__=="__main__":
    os.makedirs(CONFIG.OUTPUT_DIR,exist_ok=True)
    client=NSEClient()
    with open(CONFIG.SYMBOLS_FILE) as f:
        syms=[s.strip().upper() for s in f if s.strip()]
    rows=[]
    for sym in syms:
        seg=infer_segment(sym)
        try:
            raw=client.fetch(sym,seg)
            rep=analyze(raw,sym,seg,CONFIG.EXPIRY_PREF)
            text=build_report(rep)
            print(text)
            rows.append(dict(
                symbol=rep.symbol,segment=rep.segment,expiry=rep.expiry,as_of=rep.asof,
                pcr=round(rep.pcr,3),total_call_oi=rep.total_call_oi,total_put_oi=rep.total_put_oi,
                delta_call_oi=rep.delta_call_oi,delta_put_oi=rep.delta_put_oi,
                top5_resistances=fmt_pairs(rep.call_top5),top5_supports=fmt_pairs(rep.put_top5),
                res_gaps=fmt_list(rep.res_gaps),sup_gaps=fmt_list(rep.sup_gaps),
                ce_hotspot=rep.ce_hot,pe_hotspot=rep.pe_hot,
                equilibrium=rep.equilibrium,equilibrium_strike=rep.eq_strike,
                sentiment=rep.sentiment,rationale=rep.rationale,
                report_text=text,status="ok"
            ))
        except Exception as e:
            print(f"[WARN] {sym}: {e}")
            rows.append(dict(symbol=sym,segment=seg,status=f"error: {e}"))
        time.sleep(CONFIG.REQUEST_SLEEP_SEC)

    summary_path=os.path.join(CONFIG.OUTPUT_DIR,"summary.csv")
    pd.DataFrame(rows).to_csv(summary_path,index=False)
    print(f"\n✅ Summary saved to {summary_path} (fresh each run)")


Symbol: 360ONE | Segment: equities | Expiry: 25-Nov-2025
As Of : 03-Nov-2025 15:30:00
------------------------------------------------------------------------
Total Call OI: 2,211
Total Put  OI: 1,116
PCR        : 0.50
ΔCall OI   : 209
ΔPut  OI   : -31
------------------------------------------------------------------------
Top-5 Resistances (CE OI): 1200:678; 1180:408; 1100:300; 1160:243; 1140:152
Top-5 Supports    (PE OI): 1100:242; 1000:182; 1140:143; 1020:99; 1200:70
Flow (ΔOI) CE hotspot: 1100 | ΔOI: 135
Flow (ΔOI) PE hotspot: 1220 | ΔOI: 6
Closest resistance gaps: 20; 20; 20
Closest support gaps   : 20; 40; 60
------------------------------------------------------------------------
Sentiment: Bearish
Why      : PCR 0.50 (<0.80) → bearish tilt; ΔCallOI 209 > -31 → bearish tilt
Symbol: ABB | Segment: equities | Expiry: 25-Nov-2025
As Of : 03-Nov-2025 15:30:00
------------------------------------------------------------------------
Total Call OI: 5,487
Total Put  OI: 4,934
PCR      